In [21]:
from lxml import etree
import os.path
import os
import fnmatch
from numpy import cumsum
from operator import itemgetter
import numpy as np
import itertools
from matplotlib.pyplot import *
from textwrap import wrap
import numpy as np
from sklearn.metrics import classification_report
import random
import re
import csv
import matplotlib.pylab as plt
import numpy as np

In [2]:
print('This software generates a figure of profile clusters on a selected corpus \nusage: --dir path/to/your/source/dir --figname name_for_your_fig')
parser = argparse.ArgumentParser()
parser.add_argument('--dir', help= '/your/directory/to/tagged/files/')
parser.add_argument('--figname', help= 'name_for_your_fig')
args = parser.parse_args()
if len(sys.argv) == 1:
    sys.exit()

This software generates a figure of profile clusters on a selected corpus 
usage: --dir path/to/your/source/dir --figname name_for_your_fig


NameError: name 'argparse' is not defined

In [ ]:
argsdir= os.path.join(args.dir, '')
argsfig= os.path.join(args.figname, '')
files_list=fnmatch.filter(os.listdir(path_to_folder), '*.xml')

In [22]:
path_to_folder = "/home/odysseus/Bureau/ANR/code/CurveMatching/tagged_zola/"
argsfig='clusters'
directory_dataset='dataset/'

In [4]:
if not os.path.exists(directory):
    os.makedirs(directory)

NameError: name 'directory' is not defined

In [23]:
files_list=fnmatch.filter(os.listdir(path_to_folder), '*.xml')

In [6]:
def euclid_dist(t1,t2):
    return np.sqrt(sum((t1-t2)**2))

In [7]:
def DTWDistance(s1, s2):
    DTW={}
    
    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [8]:
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [9]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return np.sqrt(LB_sum)

In [10]:
def knn(train,test,w):
    preds=[]
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    return classification_report(test[:,-1],preds)

In [11]:
def k_means_clust(data,names, num_clust,num_iter,w):  
    counter=0
    centroids=random.sample(list(data),num_clust)
    
    for n in range(num_iter):
        counter+=1
        assignments={}
        mapClust={}
        #assign data points to clusters
        for ind,i in enumerate(data):
            min_dist=float('inf')
            closest_clust=None
            for c_ind,j in enumerate(centroids):
                if LB_Keogh(i,j,5)<min_dist:
                    cur_dist=DTWDistance(i,j,w)
                    if cur_dist<min_dist:
                        min_dist=cur_dist
                        closest_clust=c_ind
            if closest_clust in assignments:
                assignments[closest_clust].append(ind)
            else:
                assignments[closest_clust]=[]
    
        #recalculate centroids of clusters
        for key in assignments:
            nameString=''
            clust_sum=0
            for k in assignments[key]:
                clust_sum=clust_sum+data[k]
                name=names[k].replace('_',' ')
                if name not in nameString:
                    nameString+=name+' // '
            centroids[key]=[m/len(assignments[key]) for m in clust_sum]
            mapClust[nameString]=centroids[key]
        
    return mapClust

In [12]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [24]:
chaps=list()
names=list()
data_per_name=list()

for file in files_list:
    print(file)
    global_nb_words = 0
    tmpFile=file.replace("/",":")
    full_path=path_to_folder+tmpFile
    if os.path.isfile(full_path):
        tree=etree.parse(full_path)
        nb_words_per_chap=list()
        if tree.findall(".//div[@type='chapter']"):
            nb_chaps = len(tree.findall(".//div[@type='chapter']"))
            for idx, chap in enumerate(tree.findall(".//div[@type='chapter']")):
                nb_words = len(chap.findall(".//word"))
                global_nb_words+= nb_words
                nb_words_per_chap.append(nb_words)
                print(chap)
                print(nb_words_per_chap)
            print(nb_chaps)
            list_perc_per_chap=list()
            cum_l = np.cumsum(nb_words_per_chap)
            for idx,num in enumerate(cum_l):
                if idx>0:
                    cur_num = (num/global_nb_words)*100
                    prev_num = (cum_l[idx-1]/global_nb_words)*100
                    list_perc_per_chap.append(cur_num - prev_num)
                else:
                    list_perc_per_chap.append((num/global_nb_words)*100)
              
            num_div = 101/len(list_perc_per_chap)
            
            perc_chap_100=dict()
            val=0
            num_div_prev=num_div
            
            index=0
            for i in range(0,101):      
                if i > num_div_prev:
                    num_div_prev+=num_div
                    index+=1
                perc_chap_100[i]=(list_perc_per_chap[index]/num_div)
            
            names.append(re.sub(u'\n','',tree.find(".//title").text).replace("     ","").replace(" ","_"))
            
            print(perc_chap_100)
            
            data_per_name.append(perc_chap_100.values())          
            
        else :
            print("!!! No chapter found in file : "+file)         

with open('datasets/names.txt', 'w') as f:
    f.write((' '.join(names*4)))
    f.close()
writer=csv.writer(open('datasets/data.csv','w'),delimiter='\t')
for i in range (0,4):
    for percs in data_per_name:
        writer.writerow(percs)

1872_Zola-Emile_La-curee.xml
<Element div at 0x7f1b3e51aec8>
[17847]
<Element div at 0x7f1b3e51abc8>
[17847, 20611]
<Element div at 0x7f1b3e51a888>
[17847, 20611, 17036]
<Element div at 0x7f1b3e51a288>
[17847, 20611, 17036, 22536]
<Element div at 0x7f1b3e51ab48>
[17847, 20611, 17036, 22536, 21770]
<Element div at 0x7f1b3e51ac48>
[17847, 20611, 17036, 22536, 21770, 19596]
<Element div at 0x7f1b3e51ab08>
[17847, 20611, 17036, 22536, 21770, 19596, 8929]
7
{0: 0.96389697415095099, 1: 0.96389697415095099, 2: 0.96389697415095099, 3: 0.96389697415095099, 4: 0.96389697415095099, 5: 0.96389697415095099, 6: 0.96389697415095099, 7: 0.96389697415095099, 8: 0.96389697415095099, 9: 0.96389697415095099, 10: 0.96389697415095099, 11: 0.96389697415095099, 12: 0.96389697415095099, 13: 0.96389697415095099, 14: 0.96389697415095099, 15: 1.1131775947904547, 16: 1.1131775947904547, 17: 1.1131775947904547, 18: 1.1131775947904547, 19: 1.1131775947904547, 20: 1.1131775947904547, 21: 1.1131775947904547, 22: 1.113

<Element div at 0x7f1b3e7fb288>
[543]
<Element div at 0x7f1b3e7fb548>
[543, 12684]
<Element div at 0x7f1b3e7fb348>
[543, 12684, 13329]
<Element div at 0x7f1b3e7fb488>
[543, 12684, 13329, 14218]
<Element div at 0x7f1b3e7fb8c8>
[543, 12684, 13329, 14218, 13781]
<Element div at 0x7f1b3e7fbec8>
[543, 12684, 13329, 14218, 13781, 15697]
<Element div at 0x7f1b3e7fbb88>
[543, 12684, 13329, 14218, 13781, 15697, 14776]
<Element div at 0x7f1b3e7fbd08>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16664]
<Element div at 0x7f1b3e7fb2c8>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16664, 17364]
<Element div at 0x7f1b3e7fbc88>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16664, 17364, 17957]
<Element div at 0x7f1b3e7fbfc8>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16664, 17364, 17957, 17436]
<Element div at 0x7f1b3e7fbf88>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16664, 17364, 17957, 17436, 18652]
<Element div at 0x7f1b3e7fb988>
[543, 12684, 13329, 14218, 13781, 15697, 14776, 16

<Element div at 0x7f1b3e362d48>
[11782]
<Element div at 0x7f1b3e362448>
[11782, 9656]
<Element div at 0x7f1b3e362608>
[11782, 9656, 9102]
<Element div at 0x7f1b3e362808>
[11782, 9656, 9102, 13780]
<Element div at 0x7f1b3e57be08>
[11782, 9656, 9102, 13780, 14609]
<Element div at 0x7f1b3e57b808>
[11782, 9656, 9102, 13780, 14609, 14479]
<Element div at 0x7f1b3e57b0c8>
[11782, 9656, 9102, 13780, 14609, 14479, 13141]
<Element div at 0x7f1b3e57b548>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 15988]
<Element div at 0x7f1b3e57b848>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 15988, 10964]
<Element div at 0x7f1b3e57b888>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 15988, 10964, 13431]
<Element div at 0x7f1b3e57be48>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 15988, 10964, 13431, 15093]
<Element div at 0x7f1b3e57b788>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 15988, 10964, 13431, 15093, 9793]
<Element div at 0x7f1b3e57bb08>
[11782, 9656, 9102, 13780, 14609, 14479, 13141, 

<Element div at 0x7f1b3e54a808>
[502]
<Element div at 0x7f1b3e54a288>
[502, 14436]
<Element div at 0x7f1b3e54a8c8>
[502, 14436, 16958]
<Element div at 0x7f1b3e54a048>
[502, 14436, 16958, 17614]
<Element div at 0x7f1b3e54aac8>
[502, 14436, 16958, 17614, 22286]
<Element div at 0x7f1b3e54a188>
[502, 14436, 16958, 17614, 22286, 27725]
<Element div at 0x7f1b3e54a508>
[502, 14436, 16958, 17614, 22286, 27725, 33168]
<Element div at 0x7f1b3e54af48>
[502, 14436, 16958, 17614, 22286, 27725, 33168, 9566]
8
{0: 0.027951478849688068, 1: 0.027951478849688068, 2: 0.027951478849688068, 3: 0.027951478849688068, 4: 0.027951478849688068, 5: 0.027951478849688068, 6: 0.027951478849688068, 7: 0.027951478849688068, 8: 0.027951478849688068, 9: 0.027951478849688068, 10: 0.027951478849688068, 11: 0.027951478849688068, 12: 0.027951478849688068, 13: 0.8037998977571652, 14: 0.8037998977571652, 15: 0.8037998977571652, 16: 0.8037998977571652, 17: 0.8037998977571652, 18: 0.8037998977571652, 19: 0.8037998977571652, 20

In [26]:
num_clusters=5
num_iters=50
w=20

In [27]:
plt.rcParams["figure.figsize"] = [16,9]
test = np.genfromtxt('datasets/data.csv', delimiter='\t')
with open('datasets/names.txt', 'r') as namesFile:
    stringNames=namesFile.read()
namesArray=stringNames.split(' ')

mapOfTimeSeries=k_means_clust(test,namesArray,num_clusters,num_iters,w)
values=list(mapOfTimeSeries.values())
keys=list(mapOfTimeSeries.keys())
fig=plt.figure()
for idx,i in enumerate(mapOfTimeSeries.values()):

    keys = [ '\n'.join(wrap(l, 50)) for l in keys ]
    key = keys[idx]
    p=plt.plot(movingaverage(i, 1),label=key)
    legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=1,
       ncol=2, mode="expand", borderaxespad=0.)


fig.savefig(argsfig+'.png')
plt.close(fig)